# BERT Training

BERT is a pretrained NLP vectorization model. 

However, since I am wanting to use it for sentiment analysis, I need to add a classifier on top. 

This implementation will be used to calculate the sentiment (pos/neut/neg) on a corpus of newspaper articles.

Unfortunately, I couldn't find any news articles that had already been labeled, so I'm going to train the classifier with data from Reddit and Twitter.copyright

Not ideal, but it will be closer than training it on IMDB reviews or something

In [2]:
# background stuff
import numpy as np
import pandas as pd

# tf stuff
import tensorflow as tf
import tensorflow_hub as hub

### Load in the bert models

In [3]:
# preprocessing setup
preprocess = hub.load('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')

# pretrained model
bert = hub.load('https://tfhub.dev/googl/experts/bert/wiki_books/sst2/2')

FileNotFoundError: Op type not registered 'CaseFoldUTF8' in binary running on NO2. Make sure the Op and Kernel are registered in the binary running in this process. Note that if you are loading a saved graph which used ops from tf.contrib, accessing (e.g.) `tf.contrib.resampler` should be done before importing the graph, as contrib ops are lazily registered when the module is first accessed.
 You may be trying to load on a different device from the computational device. Consider setting the `experimental_io_device` option in `tf.saved_model.LoadOptions` to the io_device such as '/job:localhost'.